# Data Wrangling with Spark SQL Quiz

This quiz uses the same dataset and most of the same questions from the earlier "Quiz - Data Wrangling with Data Frames Jupyter Notebook." For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType

spark = SparkSession.builder.appName('Data Wrangling').getOrCreate()

path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

user_log.createOrReplaceTempView('user_logdata')


# Question 1

Which page did user id ""(empty string) NOT visit?

In [6]:

spark.sql(
'''

select distinct page from user_logdata
minus
select distinct page from user_logdata 
where userId=''

''').show()

+----------------+
|            page|
+----------------+
|Submit Downgrade|
|       Downgrade|
|          Logout|
|   Save Settings|
|        Settings|
|        NextSong|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+



# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

I will prefer SQL becuase it is similar python and SQL so it is easy to code and it will be understandable by larger group of people.

# Question 3

How many female users do we have in the data set?

In [8]:

spark.sql(
'''

select count(distinct userID) page from user_logdata
where gender='F'

''').show()

+----+
|page|
+----+
| 462|
+----+



# Question 4

How many songs were played from the most played artist?

In [11]:

spark.sql(
'''

select artist,count(song) num_song  from user_logdata 
group by artist order by num_song desc

''').show(1)

+--------+--------+
|  artist|num_song|
+--------+--------+
|Coldplay|      83|
+--------+--------+
only showing top 1 row



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [17]:
# Add column home 
is_home = spark.sql(
'''

select userID, page, ts, case when page='Home' then 1 else 0 end home from user_logdata 
where page in ('NextSong','Home')

''')

In [28]:
# keep the results in a view
is_home.createOrReplaceTempView("is_home_table")

In [30]:
# Cumulative sum over home column
cumulative_sum = spark.sql("SELECT *, SUM(home) OVER \
    (PARTITION BY userID ORDER BY ts DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS period \
    FROM is_home_table")

In [31]:
# keep the results in a view
cumulative_sum.createOrReplaceTempView("period_table")

# find the average count for NextSong
spark.sql("SELECT AVG(count_results) FROM \
          (SELECT COUNT(*) AS count_results FROM period_table \
GROUP BY userID, period, page HAVING page = 'NextSong') AS counts").show()

+------------------+
|avg(count_results)|
+------------------+
| 6.898347107438017|
+------------------+

